<a href="https://colab.research.google.com/github/AnudeepDasAD/Expression-Detection/blob/master/Expression_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Need to collect images first
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#PIL is the Python Image Library
from keras import regularizers

import h5py

hf = h5py.File('expression_train.h5', 'r')
hr = h5py.File('expression_test.h5', 'r')

x_train = hf.get('x_train')
y_labels = hf.get('y_labels')

x_test = hr.get('x_test')
y_test = hr.get('y_test')

labels = ["Neutral", "Anger", "Contempt", "Disgust", "Fear", "Happy", "Sadness", "Surprise"]

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

'''
#Actual folder names
labels_directory = "Emotion"

images_directory= "cohn-kanade-images"

#label will be the folder's name

current_id=0
label_ids = {"Neutral":0, "Anger":1, "Contempt":2, "Disgust":3, "Fear":4, "Happy":5, "Sadness":6, "Surprise": 7}


#Will hold the paths to the labels because the file structure for the images is very similar
paths = []

#Walking through the directory and assembling all of the y-values
for root, dirs, files in os.walk(labels_directory):
	#print("In forst loop")
	for file in files:
		if file.endswith("txt"):
			path = os.path.join(root, file)
			paths.append(path)
			#f = open(str(path), "r")
			with open(str(path)) as f:
				for line in f:
					line = line.strip()
					#Line is not empty
					if line:
						#print(int(float(line))) It works!
						id_ = int(float(line))
						y_labels.append(id_)
						#print(id_)

#Assembling the images and x_train

index = 0
for path in paths:
	#The first part of the relative path changes
	path = path.replace(labels_directory, images_directory)

	#The name of 
	path = path.replace("_emotion.txt", ".png")

	image_array = Image.open(path)
	#pil_image.resize((275,275), Image.ANTIALIAS)
	#Convert the image into a numpy array
  
	image_array = np.array(image_array, "uint8")


	#print(image_array.shape)

	#Ensuring all have the same shape   
	if image_array.shape != (490,640):
		y_labels.pop(index)
		continue
        
	x_train.append(image_array)

	index += 1
		

#print(x_train.shape)

x_train = np.array(x_train)
x_train = x_train.reshape(-1,490,640,1)

'''
print(x_train.shape)
print(y_labels.shape)

model = tf.keras.Sequential([
#Convolutions are filters made to detect certain aspects of images (some edges)
   tf.keras.layers.Conv2D(60, kernel_size=(3,3), input_shape=(300,300,1)),
    tf.keras.layers.Dense(40, kernel_regularizer=regularizers.l2(0.001), activation=tf.nn.relu, input_shape=(300,300,1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(50, kernel_size=(3,3), input_shape=(300,300,1)),
    #tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Dense(40, kernel_regularizer=regularizers.l2(0.001), activation=tf.nn.relu, input_shape=(300,300,1)),
  
    tf.keras.layers.Dense(30, activation=tf.nn.relu),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation=tf.nn.softmax)
	])

#print(len(x_train))
#print(len(y_labels))
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy")

history = model.fit(x=x_train, y=y_labels, epochs=10, validation_split=0.1, shuffle="batch")

#plt.imshow(x_test[11].reshape(300,300))
pred = model.predict(x_train[11].reshape(1,300,300,1))
print(labels[np.argmax(pred)])

model.evaluate(x=x_test, y=y_test)

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
#length of training loss is total number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()




/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(261, 300, 300, 1)
(261,)
Train on 234 samples, validate on 27 samples
Epoch 1/10
234/234 [==============================] - 7s 32ms/sample - loss: 152.0613 - val_loss: 238.0932
Epoch 2/10
234/234 [==============================] - 5s 20ms/sample - loss: 81.0447 - val_loss: 142.7365
Epoch 3/10
234/234 [==============================] - 5s 20ms/sample - loss: 63.9248 - val_loss: 96.2076
Epoch 4/10
234/234 [==============================] - 5s 20ms/sample - loss: 32.0933 - val_loss: 31.6028
Epoch 5/10
192/234 [=======================>......] - ETA: 0s - loss: 21.5397

KeyboardInterrupt: ignored